In [1]:
# preamble to be able to run notebooks in Jupyter and Colab
try:
    from google.colab import drive
    import sys
    
    drive.mount('/content/drive')
    notes_home = "/content/drive/Shared drives/CSC310/notes/"
    user_home = "/content/drive/My Drive/"
    
    sys.path.insert(1,notes_home) # let the notebook access the notes folder

except ModuleNotFoundError:
    notes_home = "" # running native Jupyter environment -- notes home is the same as the notebook
    user_home = ""  # under Jupyter we assume the user directory is the same as the notebook

# NLP & ML

## KNN

In [2]:
import pandas
import numpy as np
np.set_printoptions(formatter={'float_kind':"{:3.2f}".format})
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import cross_val_score
from re import sub
from nltk.stem import PorterStemmer


print("******** setup **********")
stemmer = PorterStemmer()

# get the newsgroup database
cats = ['talk.politics.misc', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train', 
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=cats)

# extract into dataframes
text = pandas.DataFrame(newsgroups_train.data, columns=['text'])
label = pandas.DataFrame(newsgroups_train.target, columns=['label'])['label'].apply(lambda x: cats[x])

******** setup **********


/Users/lutz/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.datasets.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
text.iloc[3,0]

'Ahhh, remember the days of Yesterday?  When we were only \n\tgoing to pay $17 / month?\n\n\tWhen only 1.2% of the population would pay extra taxes?\n\n\tRemember when a few of us predicted that it wasn\'t true?  :)\n\tRemember the Inaugural?   Dancing and Singing!  Liberation\n\tat last!  \n\n\tWell, figure *this* out:\n\n\t5% VAT, estimated to raise $60-100 Billion per year ( on CNN )\n\tWork it out, chum...\n\n\t     $60,000,000,000  /  125,000,000 taxpayers = $480 / year\n\n        But, you exclaim, " I\'ll get FREE HEALTH CARE! "\n\tBut, I exclaim, " No, you won\'t! "\n\n\tThis is only for that poor 37 million who have none.  Not for\n\tYOU, chum. :)  That comes LATER.\n\n\tAdd in the estimates of the energy tax costs - $300-500 / year\n\n\tPlus, all that extra "corporate and rich" taxes that will \n\ttrickle down, and what do you have?\n\n\t$1,000 / year, just like I said two months ago.\n\n\tAnd, the best part?   You don\'t GET ANYTHING for it.\n\n\tDeficit is STILL projected to

In [4]:
label.head()

0    sci.space
1    sci.space
2    sci.space
3    sci.space
4    sci.space
Name: label, dtype: object

In [5]:
print("******** prepare data **********")
new_data = []
for i in range(text.shape[0]):
    new_data.append(sub("[^a-zA-Z]", " ", text.iloc[i,0]))

lowercase_data = []
for i in range(len(new_data)):
    lowercase_data.append(new_data[i].lower())

stemmed_data = []
for i in range(len(lowercase_data)):
    words = lowercase_data[i].split()
    stemmed_words = []
    for w in words:
        stemmed_words.append(stemmer.stem(w))
    stemmed_data.append(" ".join(stemmed_words))

print("******** setup vector model **********")
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word", binary = True, min_df=2, stop_words='english')
docarray = vectorizer.fit_transform(stemmed_data).toarray()

print("******** model and XV **********")
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=4)

# do the 10-fold cross validation
scores = cross_val_score(model, docarray, label, cv=10)
print("Fold Accuracies: {}".format(scores))
print("XV Accuracy: {:3.2f}".format(scores.mean()))

******** prepare data **********
******** setup vector model **********
******** model and XV **********
Fold Accuracies: [0.53 0.52 0.50 0.56 0.54 0.46 0.49 0.49 0.56 0.51]
XV Accuracy: 0.52


## Naive Bayes

In [6]:
import pandas
import numpy as np
np.set_printoptions(formatter={'float_kind':"{:3.2f}".format})
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import cross_val_score
from re import sub
from nltk.stem import PorterStemmer


print("******** setup **********")
stemmer = PorterStemmer()

# get the newsgroup database
cats = ['talk.politics.misc', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train', 
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=cats)

# extract into dataframes
text = pandas.DataFrame(newsgroups_train.data, columns=['text'])
label = pandas.DataFrame(newsgroups_train.target, columns=['label'])['label'].apply(lambda x: cats[x])

print("******** prepare data **********")
new_data = []
for i in range(text.shape[0]):
    new_data.append(sub("[^a-zA-Z]", " ", text.iloc[i,0]))

lowercase_data = []
for i in range(len(new_data)):
    lowercase_data.append(new_data[i].lower())

stemmed_data = []
for i in range(len(lowercase_data)):
    words = lowercase_data[i].split()
    stemmed_words = []
    for w in words:
        stemmed_words.append(stemmer.stem(w))
    stemmed_data.append(" ".join(stemmed_words))

print("******** setup vector model **********")
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word", binary = True, min_df=2, stop_words='english')
docarray = vectorizer.fit_transform(stemmed_data).toarray()
coords = vectorizer.get_feature_names()
docterm = pandas.DataFrame(data=docarray,columns=coords)

print("******** model and XV **********")
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()


# do the 10-fold cross validation
scores = cross_val_score(model, docarray, newsgroups_train.target, cv=10)
print("Fold Accuracies: {}".format(scores))
print("XV Accuracy: {:3.2f}".format(scores.mean()))

******** setup **********
******** prepare data **********
******** setup vector model **********
******** model and XV **********
Fold Accuracies: [0.92 0.95 0.93 0.96 0.92 0.96 0.92 0.85 0.90 0.91]
XV Accuracy: 0.93
